# Imports


In [104]:
import pandas as pd
from requests import get
from bs4 import BeautifulSoup as soupify

import acquire
from importlib import reload

# 2. News Articles

We will now be scraping text data from inshorts, a website that provides a brief overview of many different topics.

Write a function that scrapes the news articles for the following topics:

    > Business
    > Sports
    > Technology
    > Entertainment
    
The end product of this should be a function named get_news_articles that returns a list of dictionaries, where each dictionary has this shape:

    {
        'title': 'The article title',
        'content': 'The article content',
        'category': 'business' # for example
    }    

> Start by inspecting the website in your browser. Figure out which elements will be useful.
    

In [33]:
url='https://inshorts.com/en/read'
soup=soupify(get(url).content)


In [36]:
def get_cats (base_url):
    """ 
    Purpose:
        
    ---
    Parameters:
        
    ---
    Returns:
    
    """
    soup = soupify(get(base_url).content)
    return [category.text.lower() for category in soup.find_all('li')[1:]]

> Start by creating a function that handles a single article and produces a dictionary like the one above.
    

In [34]:
soup.find_all('li')[1].text.lower()

'india'

In [37]:
get_cats(url)

['india',
 'business',
 'sports',
 'world',
 'politics',
 'technology',
 'startup',
 'entertainment',
 'miscellaneous',
 'hatke',
 'science',
 'automobile']

In [35]:
url + '/' + soup.find_all('li')[1].text.lower()

'https://inshorts.com/en/read/india'

> Next create a function that will find all the articles on a single page and call the function you created in the last step for every article on the page.
    

In [38]:
cat_url = url + '/' + 'science'

In [39]:
cat_soup = soupify(get(cat_url).content)



In [40]:
cat_soup.find_all('span', itemprop='headline')[0].text


'New species of beetle named after Novak Djokovic'

> Now create a function that will use the previous two functions to scrape the articles from all the pages that you need, and do any additional processing that needs to be done.
    
    
    


In [97]:
def get_news_articles(fresh=False, base_url='https://inshorts.com/en/read'):
    """ 
    Purpose:
        
    ---
    Parameters:
        
    ---
    Returns:
    
    """
    if fresh == False:
        all_articles = pd.read_csv('news_articles.csv')
        return all_articles

    #categories = get_cats(base_url)
    categories = ['business', 'sports', 'technology',  'entertainment']
    all_articles = []

    for category in categories:
        url = base_url + '/' + category
        soup = soupify(get(url).content)
        titles = [title.text for title in soup
                    .find_all('span', itemprop='headline')]
        bodies = [body.text for body in soup
                    .find_all('div', itemprop='articleBody')]
        articles = [{'title': title, 
                        'category': category,
                        'body': body} for title, body in zip(
                            titles, bodies
                        )]
        all_articles.extend(articles)
    
    all_articles = pd.DataFrame(all_articles)
    all_articles.to_csv('news_articles.csv', index=False)

    return all_articles

In [101]:
all_articles = get_news_articles()

In [102]:
all_articles

,title,category,body
0,Nikhil Chinapa shares video of cockroach on Vi...,business,DJ Nikhil Chinapa has asked Vistara airline to...
1,"Russia can destroy US, Europe within 30 minute...",business,The world's richest person Elon Musk on Saturd...
2,'Make-in-India' record-breaker: Piyush Goyal a...,business,Union Commerce Minister Piyush Goyal lauded th...
3,What are the rates at which employees are leav...,business,Infosys has reported an attrition rate of 27.1...
4,"Tata Power hit by cyberattack, says 'all criti...",business,Tata Power has confirmed that it was hit by a ...
...,...,...,...
95,Film on RSS founder KB Hedgewar in the works,entertainment,\nA biopic on RSS founder KB Hedgewar is in ea...
96,Filmmaker Vishal Bhardwaj buys new luxury apar...,entertainment,Vishal Bhardwaj has reportedly purchased a new...
97,It's an Indian spy-thriller without bikinis & ...,entertainment,"Actor Karthi, while speaking about his upcomin..."
98,Was told 'Why're you dancing? Look after the h...,entertainment,Actress Madhuri Dixit in an interview spoke ab...


## Bonus: cache the data

Write your code such that the acquired data is saved locally in some form or fashion. Your functions that retrieve the data should prefer to read the local data instead of having to make all the requests everytime the function is called. Include a boolean flag in the functions to allow the data to be acquired "fresh" from the actual sources (re-writing your local cache).

In [103]:
all_articles.to_csv('news_articles.csv', index=False)

In [52]:
all_articles

,title,category,body
0,Nikhil Chinapa shares video of cockroach on Vi...,business,DJ Nikhil Chinapa has asked Vistara airline to...
1,'Make-in-India' record-breaker: Piyush Goyal a...,business,Union Commerce Minister Piyush Goyal lauded th...
2,"Tata Power hit by cyberattack, says 'all criti...",business,Tata Power has confirmed that it was hit by a ...
3,What are the rates at which employees are leav...,business,Infosys has reported an attrition rate of 27.1...
4,"Russia can destroy US, Europe within 30 minute...",business,The world's richest person Elon Musk on Saturd...
...,...,...,...
95,Filmmaker Vishal Bhardwaj buys new luxury apar...,entertainment,Vishal Bhardwaj has reportedly purchased a new...
96,"John is a gentleman, was a pleasure to work wi...",entertainment,"Actress Manushi Chhillar, speaking about her u..."
97,I'm happy being single: Rajev on reports sayin...,entertainment,"Actor Rajev Paul, ex-husband of actress Delnaa..."
98,It's an Indian spy-thriller without bikinis & ...,entertainment,"Actor Karthi, while speaking about his upcomin..."


In [51]:
df = pd.read_csv('news_articles.csv')
df

,title,category,body
0,Nikhil Chinapa shares video of cockroach on Vi...,business,DJ Nikhil Chinapa has asked Vistara airline to...
1,'Make-in-India' record-breaker: Piyush Goyal a...,business,Union Commerce Minister Piyush Goyal lauded th...
2,"Tata Power hit by cyberattack, says 'all criti...",business,Tata Power has confirmed that it was hit by a ...
3,What are the rates at which employees are leav...,business,Infosys has reported an attrition rate of 27.1...
4,"Russia can destroy US, Europe within 30 minute...",business,The world's richest person Elon Musk on Saturd...
...,...,...,...
95,Filmmaker Vishal Bhardwaj buys new luxury apar...,entertainment,Vishal Bhardwaj has reportedly purchased a new...
96,"John is a gentleman, was a pleasure to work wi...",entertainment,"Actress Manushi Chhillar, speaking about her u..."
97,I'm happy being single: Rajev on reports sayin...,entertainment,"Actor Rajev Paul, ex-husband of actress Delnaa..."
98,It's an Indian spy-thriller without bikinis & ...,entertainment,"Actor Karthi, while speaking about his upcomin..."


In [105]:
reload(acquire)

<module 'acquire' from '/Users/sinao/codeup-data-science/natural-language-processing-exercises/acquire.py'>

In [107]:
df = acquire.get_news_articles()
df

,title,category,body
0,Nikhil Chinapa shares video of cockroach on Vi...,business,DJ Nikhil Chinapa has asked Vistara airline to...
1,"Russia can destroy US, Europe within 30 minute...",business,The world's richest person Elon Musk on Saturd...
2,'Make-in-India' record-breaker: Piyush Goyal a...,business,Union Commerce Minister Piyush Goyal lauded th...
3,What are the rates at which employees are leav...,business,Infosys has reported an attrition rate of 27.1...
4,"Tata Power hit by cyberattack, says 'all criti...",business,Tata Power has confirmed that it was hit by a ...
...,...,...,...
95,Film on RSS founder KB Hedgewar in the works,entertainment,\nA biopic on RSS founder KB Hedgewar is in ea...
96,Filmmaker Vishal Bhardwaj buys new luxury apar...,entertainment,Vishal Bhardwaj has reportedly purchased a new...
97,It's an Indian spy-thriller without bikinis & ...,entertainment,"Actor Karthi, while speaking about his upcomin..."
98,Was told 'Why're you dancing? Look after the h...,entertainment,Actress Madhuri Dixit in an interview spoke ab...
